In [54]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import (
    Conv2D, Activation, BatchNormalization,
    UpSampling2D, Input, Concatenate,
    Conv2DTranspose
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
#from models import unet_model

In [55]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [56]:
tf.keras.backend.clear_session()

In [57]:
np.random.seed(42)
tf.random.set_seed(42)

# Hyperparameters

In [58]:
IMAGE_SIZE = 256
EPOCHS = 10
BATCH_SIZE = 4
LR = 1e-4

PATH = "."

# Dataset

In [59]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    
    total_size = len(images)
    valid_size = int(total_size * split)
    test_size = int(total_size * split)
    
    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)
    
    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)
    
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [60]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x / 255.0
    
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    
    return x

## Building the tf.data pipeline

In [61]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    
    return x, y


def tf_dataset(x, y, batch=BATCH_SIZE):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    
    return dataset

In [62]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(PATH)

print("Training data: ", len(train_x))
print("Validation data: ", len(valid_x))
print("Testing data: ", len(test_x))

Training data:  264
Validation data:  33
Testing data:  33


In [63]:
def read_and_rgb(x):
    x = cv2.imread(x)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    
    return x

# Model

In [64]:
#inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
#encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.35)

In [65]:
#encoder.summary()

In [66]:
#inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
#encoder2 = Xception(input_tensor=inputs, weights="imagenet", include_top=False)

In [67]:
#encoder2.summary()

In [68]:
#x = encoder2.get_layer("block3_sepconv2_act").output
#x = Conv2DTranspose(x.shape[3], (2, 2), strides=(1, 1), padding='valid') (x)
#x

In [69]:
#x = encoder2.get_layer("block13_sepconv2_act").output
#x = UpSampling2D((2, 2))(x)
#x

In [70]:
def model():
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")

    encoder = Xception(input_tensor=inputs, weights="imagenet", include_top=False)
    # skip_connection_names = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    # 8 -> 16 -> 32 -> 63 -> 125 -> 127 -> 256
    skip_connection_names = ["input_image", "block1_conv1_act", "block3_sepconv2_act", "block4_sepconv2_act", "block13_sepconv2_act"]
    fix_skip_names = ["block1_conv1_act", "block3_sepconv2_act"]
    encoder_output = encoder.get_layer("block14_sepconv2_act").output
    #skip_connection_names = ["input_image", "re_lu", "re_lu_3", "re_lu_7"]
    #encoder_output = encoder.get_layer("re_lu_16").output

    f = [16, 32, 48, 64, 80]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        x_skip = encoder.get_layer(skip_connection_names[-i]).output
        print("x_skip original: " + str(x_skip.shape))
        if skip_connection_names[-i] in fix_skip_names:
            x_skip = Conv2DTranspose(x_skip.shape[3], (2, 2), strides=(1, 1), padding='valid') (x_skip)
        print("x original" + str(x.shape))
        x = UpSampling2D((2, 2))(x)
        print("x upsampling: " + str(x.shape))
        print("x_skip convolved: " + str(x_skip.shape))
        print("---------")
        x = Concatenate()([x, x_skip])

        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs, x)
    return model

In [71]:
model = model()
model.summary()

x_skip original: (None, 16, 16, 728)
x original(None, 8, 8, 2048)
x upsampling: (None, 16, 16, 2048)
x_skip convolved: (None, 16, 16, 728)
---------
x_skip original: (None, 32, 32, 728)
x original(None, 16, 16, 80)
x upsampling: (None, 32, 32, 80)
x_skip convolved: (None, 32, 32, 728)
---------
x_skip original: (None, 63, 63, 256)
x original(None, 32, 32, 64)
x upsampling: (None, 64, 64, 64)
x_skip convolved: (None, 64, 64, 256)
---------
x_skip original: (None, 127, 127, 32)
x original(None, 64, 64, 48)
x upsampling: (None, 128, 128, 48)
x_skip convolved: (None, 128, 128, 32)
---------
x_skip original: (None, 256, 256, 3)
x original(None, 128, 128, 32)
x upsampling: (None, 256, 256, 32)
x_skip convolved: (None, 256, 256, 3)
---------
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_image (InputLayer)    [(None, 256, 

# Metrics

In [72]:
smooth = 1e-15

def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Training

In [73]:
train_dataset = tf_dataset(train_x, train_y, batch=BATCH_SIZE)
valid_dataset = tf_dataset(valid_x, valid_y, batch=BATCH_SIZE)

In [76]:
opt = tf.keras.optimizers.Nadam(LR)
metrics = [dice_coef]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

In [77]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

In [78]:
train_steps = len(train_x) // BATCH_SIZE
valid_steps = len(valid_x) // BATCH_SIZE

if len(train_x) % BATCH_SIZE != 0:
    train_steps += 1
if len(valid_x) % BATCH_SIZE != 0:
    valid_steps += 1
    
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

Epoch 1/300


2024-02-13 17:08:47.201539: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-13 17:08:47.544598: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-13 17:08:48.633346: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-13 17:08:56.630582: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 438.88MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-13 17:08:56.765398: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 438.88MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performa

66/66 [==============================] - 290s 3s/step - loss: 0.9462 - dice_coef: 0.0538 - val_loss: 0.9476 - val_dice_coef: 0.0485 - lr: 1.0000e-04
Epoch 2/300
66/66 [==============================] - 168s 3s/step - loss: 0.9300 - dice_coef: 0.0700 - val_loss: 0.9337 - val_dice_coef: 0.0613 - lr: 1.0000e-04
Epoch 3/300
66/66 [==============================] - 163s 2s/step - loss: 0.9205 - dice_coef: 0.0795 - val_loss: 0.9221 - val_dice_coef: 0.0719 - lr: 1.0000e-04
Epoch 4/300
66/66 [==============================] - 183s 3s/step - loss: 0.9157 - dice_coef: 0.0843 - val_loss: 0.9202 - val_dice_coef: 0.0735 - lr: 1.0000e-04
Epoch 5/300
66/66 [==============================] - 201s 3s/step - loss: 0.9118 - dice_coef: 0.0882 - val_loss: 0.9079 - val_dice_coef: 0.0853 - lr: 1.0000e-04
Epoch 6/300
66/66 [==============================] - 190s 3s/step - loss: 0.9084 - dice_coef: 0.0916 - val_loss: 0.9027 - val_dice_coef: 0.0901 - lr: 1.0000e-04
Epoch 7/300
66/66 [===========================

KeyboardInterrupt: 

In [ ]:
test_dataset = tf_dataset(test_x, test_y, batch=BATCH_SIZE)

test_steps = len(test_x) // BATCH_SIZE
if len(test_x) % BATCH_SIZE != 0:
    test_steps += 1
    
model.evaluate(test_dataset, steps=test_steps)

# Results

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    return x

In [ ]:
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
for i, (x, y) in enumerate(zip(test_x[:10], test_y[:10])):
    x = read_image(x)
    y = read_mask(y)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3))

    all_images = [
        x, white_line,
        mask_parse(y), white_line,
        mask_parse(y_pred)
    ]
    image = np.concatenate(all_images, axis=1)

    fig = plt.figure(figsize=(12, 12))
    a = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(image)